# Importing all the necessary packages

In [5]:
import streamlit as st
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
import sqlite3

## Using SQLite to connect to the db

In [11]:
connexion = sqlite3.connect("../data/db/vivino.db")
cursor = connexion.cursor()

## Aswering the questions with SQL queries

In [12]:
# Question no 1 - top 10 wines

cursor.execute("""
SELECT
    w.id AS wine_id,
    w.name AS wine_name,
    v.name AS vintage_name,
    w.ratings_average,
    w.ratings_count,
    v.price_euros,
    vt.rank AS toplist_rank
FROM wines w
JOIN vintages v ON w.id = v.wine_id
LEFT JOIN vintage_toplists_rankings vt ON v.id = vt.vintage_id
WHERE toplist_rank >0
ORDER BY
    v.price_euros ASC,
    w.ratings_average DESC,
    w.ratings_count DESC
LIMIT 10;
""")
cursor.fetchall()

[(11890,
  '60 Sessantanni Old Vines Primitivo di Manduria',
  'San Marzano 60 Sessantanni Old Vines Primitivo di Manduria 2018',
  4.5,
  94289,
  24.75,
  17),
 (11890,
  '60 Sessantanni Old Vines Primitivo di Manduria',
  'San Marzano 60 Sessantanni Old Vines Primitivo di Manduria 2018',
  4.5,
  94289,
  24.75,
  1),
 (11890,
  '60 Sessantanni Old Vines Primitivo di Manduria',
  'San Marzano 60 Sessantanni Old Vines Primitivo di Manduria 2018',
  4.5,
  94289,
  24.75,
  19),
 (6331780,
  'Guerriero della Terra',
  'Guerrieri Guerriero della Terra 2019',
  4.4,
  10185,
  27.4,
  1),
 (6023708,
  'Dona Ermelinda Grande Reserva',
  'Casa Ermelinda Freitas Dona Ermelinda Grande Reserva 2017',
  4.4,
  933,
  29.95,
  8),
 (6023708,
  'Dona Ermelinda Grande Reserva',
  'Casa Ermelinda Freitas Dona Ermelinda Grande Reserva 2017',
  4.4,
  933,
  29.95,
  7),
 (6023708,
  'Dona Ermelinda Grande Reserva',
  'Casa Ermelinda Freitas Dona Ermelinda Grande Reserva 2017',
  4.4,
  933,
  29.9

In [13]:
cursor.execute("""SELECT
vintages.name AS wines_names_vintage,
vintages.ratings_average AS score,
vintages.ratings_count AS review_amount,
vintages.price_euros AS price,
vintage_toplists_rankings.rank AS toplist_rank,
regions.country_code AS country_iso,
regions.name
FROM
vintage_toplists_rankings
JOIN vintages ON vintage_toplists_rankings.vintage_id = vintages.id
JOIN wines ON vintages.wine_id = wines.id
JOIN regions ON wines.region_id = regions.id
WHERE toplist_rank >0 AND price < 50
GROUP BY wines_names_vintage
ORDER BY
    score DESC,
	price ASC,
    review_amount DESC
LIMIT 10;""")
cursor.fetchall()

[('Gitana Lupi Rezerva 2017', 4.6, 1155, 36.55, 15, 'md', 'Moldova'),
 ('Tenuta Ulisse Limited Edition 10 Vendemmie',
  4.5,
  9453,
  34.2,
  5,
  'it',
  "Montepulciano d'Abruzzo"),
 ('Pago de Carraovejas Tinto 2020',
  4.5,
  2129,
  42.95,
  1,
  'es',
  'Ribera del Duero'),
 ('San Marzano 60 Sessantanni Old Vines Primitivo di Manduria 2018',
  4.4,
  1748,
  24.75,
  17,
  'it',
  'Primitivo di Manduria'),
 ('Guerrieri Guerriero della Terra 2019', 4.4, 1803, 27.4, 1, 'it', 'Marche'),
 ('Casa Ermelinda Freitas Dona Ermelinda Grande Reserva 2017',
  4.4,
  483,
  29.95,
  8,
  'pt',
  'Península de Setúbal'),
 ('Emilio Moro Malleolus 2020', 4.4, 625, 35.95, 18, 'es', 'Ribera del Duero'),
 ('Arzuaga Reserva Ribera del Duero 2018',
  4.4,
  840,
  39.55,
  1,
  'es',
  'Ribera del Duero'),
 ('Vilafonté Series M 2014', 4.4, 273, 48.0, 7, 'za', 'Paarl'),
 ('Tommasi Amarone della Valpolicella Classico 2018',
  4.4,
  1570,
  49.8,
  4,
  'it',
  'Amarone della Valpolicella Classico')]

In [63]:
# Question no 4 - user's cluster regarding keywords

cursor.execute("""
SELECT
    wines.name AS wine_name,
    keywords_wine.count AS keyword_count,
    keywords.name AS keyword_name,
    group_concat(keywords.name) AS keyword_list
FROM wines
JOIN keywords_wine ON keywords_wine.wine_id = wines.id
JOIN keywords  ON keywords.id = keywords_wine.keyword_id
WHERE 
    keyword_count > 10
    --AND keyword_list LIKE '%coffee%toast%green apple%cream%citrus%'
GROUP BY wine_name
HAVING 
    keyword_list LIKE '%coffee%'
    AND    keyword_list LIKE '%toast%'
    AND    keyword_list LIKE '%green apple%'
    AND    keyword_list LIKE '%cream%'
    AND    keyword_list LIKE '%citrus%';
    
    
""")
cursor.fetchall()

[('Belle Epoque Brut Champagne',
  399,
  'apple',
  'apple,pear,green apple,peach,apricot,white peach,yellow apple,stone fruit,melon,brioche,toast,almond,nutty,biscuit,marzipan,hazelnut,toasted almond,walnut,vanilla,caramel,smoke,coffee,citrus,lemon,grapefruit,lime,orange,lemon peel,lemon zest,lemon curd,cream,yeast,cheese,lemon curd,toasted bread,toast,almond,butter,mushroom,toffee,butter,vanilla,caramel,pastry,toffee,oak,coffee,chocolate,brioche,almond,marzipan,hazelnut,smoke,lemon curd,honey,minerals,stone,chalk,mushroom,smoke,ginger,almond,marzipan,straw,gooseberry,almond,vanilla,ginger,honeysuckle,acacia,jasmine,elderflower,pineapple,tropical,strawberry,raspberry'),
 ('Blanc des Millénaires',
  229,
  'brioche',
  'brioche,toast,almond,nutty,hazelnut,biscuit,marzipan,walnut,caramel,vanilla,coffee,smoke,butter,caramel,vanilla,pastry,coffee,toffee,oak,brioche,almond,hazelnut,smoke,marzipan,cream,yeast,toast,butter,almond,toffee,honey,minerals,stone,chalk,smoke,saline,flint,almond,m

In [65]:
# Question 6 - Top 3 grapes

cursor.execute("""SELECT 
    grapes.name AS grape_name,
    grapes.id AS grapes_id,
    sum(most_used_grapes_per_country.wines_count) AS global_wine_count
FROM grapes
JOIN most_used_grapes_per_country ON most_used_grapes_per_country.grape_id = grapes_id
GROUP BY grape_id
ORDER BY global_wine_count DESC
LIMIT
    3;""")

cursor.fetchall()

[('Cabernet Sauvignon', 2, 9621012),
 ('Merlot', 10, 6233909),
 ('Chardonnay', 5, 3625248)]

In [66]:
cursor.execute("""SELECT
wines.name,
wines.ratings_average AS ranks,
wines.ratings_count AS review_amount,
most_used_grapes_per_country.grape_id AS grapes_id
FROM
wines
JOIN regions ON wines.region_id = regions.id
JOIN countries ON regions.country_code = countries.code
JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
WHERE 
grapes_id IN (2, 10, 5) AND
wines.ratings_count > 8000
ORDER BY wines.ratings_average DESC;""")

cursor.fetchall()

[('Special Selection Cabernet Sauvignon', 4.7, 41236, 2),
 ('Special Selection Cabernet Sauvignon', 4.7, 41236, 5),
 ('Unico Reserva Especial Edición', 4.7, 13025, 2),
 ('Unico', 4.7, 45140, 2),
 ('Amarone della Valpolicella Classico', 4.7, 8900, 5),
 ('Amarone della Valpolicella Classico', 4.7, 8900, 10),
 ('Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne', 4.7, 9564, 5),
 ('Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne', 4.7, 9564, 10),
 ('Grange', 4.7, 24356, 2),
 ('Grange', 4.7, 24356, 5),
 ('Sauternes', 4.7, 44126, 5),
 ('Sauternes', 4.7, 44126, 10),
 ('Pomerol', 4.7, 32157, 5),
 ('Pomerol', 4.7, 32157, 10),
 ('Toscana', 4.7, 16284, 5),
 ('Toscana', 4.7, 16284, 10),
 ("Tenuta Tignanello 'Solaia'", 4.6, 33143, 5),
 ("Tenuta Tignanello 'Solaia'", 4.6, 33143, 10),
 ('Cabernet Sauvignon (Signature)', 4.6, 13730, 2),
 ('Cabernet Sauvignon (Signature)', 4.6, 13730, 5),
 ('Pauillac (Premier Grand Cru Classé)', 4.6, 39283, 5),
 ('Pauillac (Premier Grand Cru Classé)', 4.6, 39283, 1